In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from helper_functions import get_local_embedding_model


load_dotenv()


llm = ChatOpenAI(model="qwen-max", temperature=0)
embedding_model = get_local_embedding_model()

# 样本数据集

In [2]:
sample_content = """保罗·格雷厄姆在2024年9月发表的文章《创始人模式》（Founder Mode）挑战了关于创业公司扩张的传统智慧。他认为，创始人应当坚持自己的独特管理风格，而非在公司成长过程中采纳传统的企业管理模式。  
传统智慧 VS 创始人模式
文章指出，针对成长型公司的传统建议——招聘优秀人才并给予他们自主权——在创业公司环境下往往行不通。这种方法虽然适用于成熟企业，但对创业公司而言可能是有害的，因为创始人的愿景和直接参与至关重要。文章提出，“创始人模式”是一种尚未被充分理解或系统化的新兴管理范式，与商学院和职业经理人推崇的“管理者模式”形成对比。  
创始人的独特能力
创始人拥有职业经理人所不具备的独特洞察力和能力，主要体现在他们对公司愿景和文化的深刻理解。格雷厄姆建议，创始人应当利用这些优势，而不是被动接受传统的管理模式。“创始人模式”虽然仍处于探索阶段，但格雷厄姆希望，随着时间推移，它能够像传统的管理者模式一样被广泛理解，从而使创始人在公司扩张的过程中依然能够保持自身的独特管理方式。  
创业公司扩张的挑战
随着创业公司的成长，许多人认为它们必须过渡到更结构化的管理方式。然而，许多创始人发现，这种转变往往会导致公司失去最初的创新精神和敏捷性，而正是这些特质促成了创业公司的成功。  
Airbnb联合创始人布莱恩·切斯基（Brian Chesky）分享了自己的经历：他曾被建议采用传统的管理模式来运营公司，但结果并不理想。最终，他找到了不同的管理方法，并受到史蒂夫·乔布斯（Steve Jobs）管理苹果公司的启发，从而取得了更好的效果。  
史蒂夫·乔布斯的管理风格
乔布斯在苹果的管理方式成为了切斯基“创始人模式”的灵感来源。其中一个显著做法是乔布斯每年都会召集苹果公司最重要的100名员工参加高层会议，而这些人并不一定处于公司的正式管理架构之中。这种非传统的方法使得苹果即便在扩张过程中仍保持类似创业公司的环境，促进了创新，并打破了层级之间的沟通壁垒。这一管理理念强调了创始人应当保持对公司运营的深度参与，而不是在公司扩张时将责任完全交给职业经理人，从而挑战了传统的管理观念。"""

In [3]:
from langchain_core.documents import Document


docs_list = [
    Document(
        page_content=sample_content,
        metadata={
            "Title": "保罗·格雷厄姆的创始人模式文章",
            "Source": "https://www.perplexity.ai/page/paul-graham-s-founder-mode-ess-t9TCyvkqRiyMQJWsHr0fnQ",
        },
    )
]

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)

for i, doc in enumerate(doc_splits):
    doc.metadata["chunk_id"] = i + 1

# 生成命题

In [5]:
from typing import List
from pydantic import BaseModel, Field


# Data model
class GeneratePropositions(BaseModel):
    """List of all the propositions in a given document"""

    propositions: List[str] = Field(
        description="List of propositions (factual, self-contained, and concise information)"
    )


# LLM with function call
structured_llm = llm.with_structured_output(GeneratePropositions)

In [6]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate


# Few shot prompting --- We can add more examples to make it good
proposition_examples = [
    {
        "document": "2003年，杨利伟乘坐神舟五号飞船成为第一个进入太空的中国人。",
        "propositions": "['杨利伟是宇航员。', '杨利伟在2003年进入太空。', '杨利伟是第一个进入太空的中国人。', '杨利伟乘坐神舟五号进入太空。', '神舟五号在2003年发射。']",
    },
]

example_proposition_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{document}"),
        ("ai", "{propositions}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_proposition_prompt,
    examples=proposition_examples,
)

In [7]:
# Prompt
system = """请将以下文本分解为简单、自成体系的命题。确保每个命题都符合以下标准：

1. 表达单一事实：每个命题应陈述一个具体事实或主张。
2. 无需上下文即可理解：命题应自成体系，即无需额外上下文即可理解。
3. 使用全名，而不是代词：避免使用代词或模棱两可的引用；使用完整的实体名称。
4. 包括相关日期/限定词：如果适用，请包括必要的日期、时间和限定词，以使事实准确。
5. 包含一个主谓关系：关注单个主语及其对应的动作或属性，没有连词或多个从句。"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        few_shot_prompt,
        ("human", "{document}"),
    ]
)

proposition_generator = prompt | structured_llm

In [8]:
propositions = []  # Store all the propositions from the document

for i in range(len(doc_splits)):
    response = proposition_generator.invoke({"document": doc_splits[i].page_content})

    # Creating proposition
    for proposition in response.propositions:
        propositions.append(
            Document(
                page_content=proposition,
                metadata={
                    "Title": "保罗·格雷厄姆的创始人模式文章",
                    "Source": "https://www.perplexity.ai/page/paul-graham-s-founder-mode-ess-t9TCyvkqRiyMQJWsHr0fnQ",
                    "chunk_id": i + 1,
                },
            )
        )

# 评估命题

In [9]:
# Data model
class GradePropositions(BaseModel):
    """Grade a given proposition on accuracy, clarity, completeness, and conciseness"""

    accuracy: int = Field(
        description="Rate from 1-10 based on how well the proposition reflects the original text."
    )

    clarity: int = Field(
        description="Rate from 1-10 based on how easy it is to understand the proposition without additional context."
    )

    completeness: int = Field(
        description="Rate from 1-10 based on whether the proposition includes necessary details (e.g., dates, qualifiers)."
    )

    conciseness: int = Field(
        description="Rate from 1-10 based on whether the proposition is concise without losing important information."
    )


# LLM with function call
structured_llm = llm.with_structured_output(GradePropositions)

In [10]:
evaluation_prompt_template = """
请根据以下标准评估以下命题：
- **Accuracy**：根据命题反映原文的程度，从 1 到 10 进行评分。
- **Clarity**：根据命题在没有额外背景的情况下理解的难易程度，从 1 到 10 进行评分。
- **Completeness**：根据命题是否包含必要的细节（例如日期、限定词）从 1 到 10 进行评分。
- **Conciseness**：根据命题是否简洁而不丢失重要信息，从 1 到 10 进行评分。

示例:
Docs: 2003年，杨利伟乘坐神舟五号飞船成为第一个进入太空的中国人。

Propositons_1: 杨利伟是宇航员。
Evaluation_1: "accuracy": 10, "clarity": 10, "completeness": 10, "conciseness": 10

Propositons_2: 杨利伟在2003年进入太空。
Evaluation_3: "accuracy": 10, "clarity": 10, "completeness": 10, "conciseness": 10

Propositons_3: 杨利伟是第一个进入太空的中国人。
Evaluation_3: "accuracy": 10, "clarity": 10, "completeness": 10, "conciseness": 10

Propositons_4: 杨利伟乘坐神舟五号进入太空。
Evaluation_4: "accuracy": 10, "clarity": 10, "completeness": 10, "conciseness": 10

Propositons_5: 神舟五号在2003年发射。
Evaluation_5: "accuracy": 10, "clarity": 10, "completeness": 10, "conciseness": 10

格式:
命题: "{proposition}"
原始文本: "{original_text}"
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", evaluation_prompt_template),
        ("human", "{proposition}, {original_text}"),
    ]
)

proposition_evaluator = prompt | structured_llm

In [11]:
# Define evaluation categories and thresholds
evaluation_categories = ["accuracy", "clarity", "completeness", "conciseness"]
thresholds = {"accuracy": 7, "clarity": 7, "completeness": 7, "conciseness": 7}


# Function to evaluate proposition
def evaluate_proposition(proposition, original_text):
    response = proposition_evaluator.invoke(
        {"proposition": proposition, "original_text": original_text}
    )

    # Parse the response to extract scores
    scores = {
        "accuracy": response.accuracy,
        "clarity": response.clarity,
        "completeness": response.completeness,
        "conciseness": response.conciseness,
    }  # Implement function to extract scores from the LLM response
    return scores


# Check if the proposition passes the quality check
def passes_quality_check(scores):
    for category, score in scores.items():
        if score < thresholds[category]:
            return False
    return True

In [12]:
evaluated_propositions = []  # Store all the propositions from the document

# Loop through generated propositions and evaluate them
for idx, proposition in enumerate(propositions):
    scores = evaluate_proposition(
        proposition.page_content,
        doc_splits[proposition.metadata["chunk_id"] - 1].page_content,
    )
    if passes_quality_check(scores):
        # Proposition passes quality check, keep it
        evaluated_propositions.append(proposition)
    else:
        # Proposition fails, discard or flag for further review
        print(f"{idx + 1}) Propostion: {proposition.page_content} \n Scores: {scores}")
        print("Fail")

2) Propostion: 文章《创始人模式》挑战了关于创业公司扩张的传统智慧。 
 Scores: {'accuracy': 8, 'clarity': 10, 'completeness': 6, 'conciseness': 10}
Fail
9) Propostion: 传统建议推荐成长型公司招聘优秀人才并给予他们自主权。 
 Scores: {'accuracy': 5, 'clarity': 10, 'completeness': 5, 'conciseness': 10}
Fail
10) Propostion: 传统建议在创业公司环境下往往行不通。 
 Scores: {'accuracy': 8, 'clarity': 9, 'completeness': 6, 'conciseness': 10}
Fail
24) Propostion: 创始人拥有独特洞察力和能力。 
 Scores: {'accuracy': 8, 'clarity': 9, 'completeness': 6, 'conciseness': 9}
Fail
36) Propostion: 创业公司面临资金管理的挑战。 
 Scores: {'accuracy': 7, 'clarity': 10, 'completeness': 6, 'conciseness': 10}
Fail
37) Propostion: 创业公司在招聘合适人才方面遇到困难。 
 Scores: {'accuracy': 7, 'clarity': 9, 'completeness': 6, 'conciseness': 9}
Fail
53) Propostion: 史蒂夫·乔布斯强调创新在他的管理实践中。 
 Scores: {'accuracy': 7, 'clarity': 8, 'completeness': 6, 'conciseness': 9}
Fail
67) Propostion: 一管理理念挑战了传统的管理观念。 
 Scores: {'accuracy': 8, 'clarity': 9, 'completeness': 4, 'conciseness': 10}
Fail


# 生成命题嵌入

In [13]:
from langchain_community.vectorstores import FAISS


vectorstore_propositions = FAISS.from_documents(evaluated_propositions, embedding_model)
retriever_propositions = vectorstore_propositions.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4},
)

In [14]:
test_query_1 = "谁的管理方式启发了 Airbnb 的布莱恩·切斯基“创始人模式”？"

res_proposition = retriever_propositions.invoke(test_query_1)

for i, r in enumerate(res_proposition):
    print(f"{i + 1}) Content: {r.page_content} --- Chunk_id: {r.metadata['chunk_id']}")

1) Content: 布莱恩·切斯基是Airbnb的联合创始人。 --- Chunk_id: 10
2) Content: 乔布斯的管理方式成为了切斯基'创始人模式'的灵感来源。 --- Chunk_id: 13
3) Content: 布莱恩·切斯基找到了不同于传统管理的方法来运营公司。 --- Chunk_id: 10
4) Content: 史蒂夫·乔布斯影响了布莱恩·切斯基对于公司管理的看法。 --- Chunk_id: 10


In [15]:
test_query_2 = "Airbnb的联合创始人是谁？"

res_proposition = retriever_propositions.invoke(test_query_2)

for i, r in enumerate(res_proposition):
    print(f"{i + 1}) Content: {r.page_content} --- Chunk_id: {r.metadata['chunk_id']}")

1) Content: 布莱恩·切斯基是Airbnb的联合创始人。 --- Chunk_id: 10
2) Content: 存在一种被称为'创始人模式'的管理方式。 --- Chunk_id: 4
3) Content: '创始人模式'正处于探索阶段。 --- Chunk_id: 6
4) Content: 格雷厄姆期望随着时间推移"创始人模式"变得像传统管理模式一样普及。 --- Chunk_id: 7


In [20]:
test_query_3 = "《创始人模式》文章是什么时候发表的？"

res_proposition = retriever_propositions.invoke(test_query_3)

for i, r in enumerate(res_proposition):
    print(f"{i + 1}) Content: {r.page_content} --- Chunk_id: {r.metadata['chunk_id']}")

1) Content: 存在一种被称为'创始人模式'的管理方式。 --- Chunk_id: 4
2) Content: '创始人模式'是一种新兴的管理范式。 --- Chunk_id: 4
3) Content: 保罗·格雷厄姆在2024年9月发表了一篇文章《创始人模式》。 --- Chunk_id: 1
4) Content: '创始人模式'正处于探索阶段。 --- Chunk_id: 6
